In [1]:
import pandas as pd
import requests
import os

In [22]:
pip install yahoo_fin

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=427b5bb53840cb36a2a7badbdb7fa39106492d444d24a7dd93eb050fabee8304
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: 

In [9]:


# Fetching stock data using Alpha Vantage API
API_KEY = os.getenv('ALPHA_KEY')
symbol = 'AAPL'  # Example: Apple Inc. stock
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_KEY}'
response = requests.get(url)
data = response.json()
# Convert to DataFrame
stock_data = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index')
stock_data = stock_data.astype(float)  # Convert to float for numerical operations
# Display the first few rows
stock_data.head()


,1. open,2. high,3. low,4. close,5. volume
2025-02-06,231.285,233.80,230.425,233.22,29925349.0
2025-02-05,228.530,232.67,228.270,232.47,39664989.0
2025-02-04,227.250,233.13,226.650,232.80,45067301.0
2025-02-03,229.990,231.83,225.700,228.01,73063301.0
2025-01-31,247.190,247.19,233.440,236.00,101075128.0


In [10]:
print([i for i in stock_data.columns])

['1. open', '2. high', '3. low', '4. close', '5. volume']


In [17]:
# Handling missing data by forward filling
stock_data.fillna(method='ffill', inplace=True)


<ipython-input-17-672e98ff14f9>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='ffill', inplace=True)


In [19]:
stock_data.rename(columns={"index": "Date", "1. open":'Open','2. high':'High', '3. low':'Low', '4. close':'Close', '5. volume':'Volume'}, inplace=True)


In [21]:
from scipy.stats import zscore
# Z-score method to identify outliers in 'Close' prices
stock_data['zscore'] = zscore(stock_data['Close'])
outliers = stock_data[stock_data['zscore'].abs() > 2]  # Values with z-score > 3
print(outliers)


              Open    High     Low   Close       Volume    zscore
2024-12-27  257.83  258.70  253.06  255.59   42355321.0  2.193650
2024-12-26  258.19  260.10  257.63  259.02   27262983.0  2.542706
2024-12-24  255.49  258.21  255.29  258.20   23234705.0  2.459259
2024-12-23  254.77  255.65  253.45  255.27   40858774.0  2.161085
2024-12-20  248.04  255.00  245.69  254.49  147495267.0  2.081708


In [14]:
df = stock_data.iloc[:100].copy()
df.reset_index(inplace=True)
df.rename(columns={"index": "Date", "1. open":'Open','2. high':'High', '3. low':'Low', '4. close':'Close', '5. volume':'Volume'}, inplace=True)

# Convert Date column to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Compute Stock Price Variations
df["Daily Return (%)"] = ((df["Close"] - df["Open"]) / df["Open"]) * 100
df["High-Low Range (%)"] = ((df["High"] - df["Low"]) / df["Low"]) * 100
df["Close-Open Diff"] = df["Close"] - df["Open"]
df["Volume Change (%)"] = df["Volume"].pct_change() * 100  # % change in volume

# Display the computed DataFrame
print("Stock price variations")
print(df.head())



Stock price variations
        Date     Open    High      Low   Close       Volume  Daily Return (%)  \
0 2025-02-06  231.285  233.80  230.425  233.22   29925349.0          0.836630   
1 2025-02-05  228.530  232.67  228.270  232.47   39664989.0          1.724062   
2 2025-02-04  227.250  233.13  226.650  232.80   45067301.0          2.442244   
3 2025-02-03  229.990  231.83  225.700  228.01   73063301.0         -0.860907   
4 2025-01-31  247.190  247.19  233.440  236.00  101075128.0         -4.526882   

   High-Low Range (%)  Close-Open Diff  Volume Change (%)  
0            1.464685            1.935                NaN  
1            1.927542            3.940          32.546454  
2            2.859034            5.550          13.619850  
3            2.715995           -1.980          62.120427  
4            5.890164          -11.190          38.339120  


In [4]:
import requests

# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
api_key = os.getenv('ALPHA_KEY')
ticker = 'AAPL'  # Example: Apple Inc.
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}'

response = requests.get(url)
data = response.json()

# Display the fetched news articles and their sentiment scores
for article in data.get('feed', []):
    print(f"Title: {article['title']}")
    print(f"Summary: {article['summary']}")
    print(f"Sentiment Score: {article['overall_sentiment_score']}\n")


Title: The DeepSeek News Makes Apple's Artificial Intelligence Strategy Look Genius
Summary: News of a new artificial intelligence ( AI ) large language model ( LLM ) in China that could rival OpenAI's ChatGPT and other top LLMs in North America has some tech investors worried. That's because DeepSeek's LLM was developed on a shoestring budget ( reported at around $6 million ) and its ...
Sentiment Score: 0.087697

Title: Alphabet's Verily Sells Granular Insurance To Elevance Health Amid Healthcare Strategy Refocus - Apple  ( NASDAQ:AAPL ) , Elevance Health  ( NYSE:ELV ) 
Summary: Verily, Alphabet Inc.'s GOOGL GOOG health technology subsidiary, announced Thursday it will sell its stop-loss insurance business, Granular Insurance Company, to Elevance Health ELV. The deal marks another strategic pivot for the Alphabet unit as it refocuses its healthcare initiatives.
Sentiment Score: 0.221236

Title: Gaming Market to grow by USD 100.8 Billion from 2025-2029, Driven by rising AR and VR adop

In [ ]:
import requests
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from textblob import TextBlob

# get yahoofin data
API_KEY = "YOUR_API_KEY"

# Function to fetch news data
def fetch_news(ticker, limit=10):
    """
    Fetch stock news  and perform sentiment analysis.

    Args:
        ticker (str): Stock ticker symbol (e.g., "AAPL").
        limit (int): Number of news articles to fetch.

    Returns:
        DataFrame: News articles with timestamp, sentiment score.
    """
    # url = f"https://api.polygon.io/v2/reference/news?ticker={ticker}&limit={limit}&apiKey={API_KEY}"
    url = base_url

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        news_list = data.get("results", [])

        # Extract relevant fields and perform sentiment analysis
        news_data = []
        for article in news_list:
            title = article.get("title", "")
            summary = article.get("description", "")
            text = f"{title} {summary}"
            sentiment_score = TextBlob(text).sentiment.polarity  # Sentiment Score (-1 to 1)

            news_data.append({
                "published_utc": article.get("published_utc"),
                "title": title,
                "summary": summary,
                "sentiment_score": sentiment_score
            })

        # Convert to DataFrame
        news_df = pd.DataFrame(news_data)
        news_df["published_utc"] = pd.to_datetime(news_df["published_utc"])
        news_df["date"] = news_df["published_utc"].dt.date  # Extract date for merging
        return news_df
    else:
        print("Error fetching news data:", response.json())
        return pd.DataFrame()

# Function to fetch stock price data from Yahoo Finance
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch historical stock prices from Yahoo Finance.

    Args:
        ticker (str): Stock ticker symbol (e.g., "AAPL").
        start_date (str): Start date for data retrieval.
        end_date (str): End date for data retrieval.

    Returns:
        DataFrame: Stock price data with daily return.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data["Daily Return (%)"] = stock_data["Adj Close"].pct_change() * 100
    stock_data["date"] = stock_data.index.date  # Extract date for merging
    return stock_data.reset_index()

# Example: Fetch data for Apple (AAPL)
ticker_symbol = "AAPL"
start_date = "2024-01-01"
end_date = "2024-02-07"

# Fetch news sentiment data
news_df = fetch_polygon_news(ticker_symbol, limit=50)

# Fetch stock price data
stock_df = fetch_stock_data(ticker_symbol, start_date, end_date)

# Merge stock price data with news sentiment scores
merged_df = stock_df.merge(news_df.groupby("date")["sentiment_score"].mean(), on="date", how="left")


print(merged_df )

# Plot Stock Price vs Sentiment Score
plt.figure(figsize=(10, 5))
plt.subplot(2, 1, 1)
plt.plot(merged_df["date"], merged_df["Adj Close"], label="Stock Price", color="blue")
plt.ylabel("Stock Price ($)")
plt.legend()

plt.subplot(2, 1, 2)
plt.bar(merged_df["date"], merged_df["sentiment_score"], label="Sentiment Score", color="green")
plt.axhline(0, color="red", linestyle="dashed")
plt.ylabel("Sentiment Score")
plt.xlabel("Date")
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()
